---

# Assignment 3

```
X1 ------\
          \
          (P) ---- (P) ---- (T)
          /
X2 ------/
         
```

1. Create a two layer neural network with one perceptron in each layer (see Diagram above). Write a validation code that does along with your implementation. The goal of network is to optimize the two perceptrons to produce the output target `T` given the inputs `X1` and `X2`. Assume the output `O` of each perceptron is

$$ O = \sigma{(w1*x1 + w2*x2 + b)} $$ where

$$\sigma(x) = \frac{1}{1+e^{-x}}$$

Feel free to change the loss function if you like.

---

## Answer
Eryu Suo 802684654

In [1]:
import numpy as np

In [2]:
class ValueNode(object):
    """
    Base object for all inputs and outputs.
    """
    def __init__(self, value, grad=0.0):
        self.value = value
        self.gradient = grad

In [3]:
class MultiplyNode(object):
    """
    Multiplies two inputs
    """
    def __init__(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = ValueNode(0.0)
        
    def forward(self):
        self.output.value = self.x1.value * self.x2.value
    
    def backward(self):
        self.x1.gradient = self.x2.value * self.output.gradient
        self.x2.gradient = self.x1.value * self.output.gradient

In [4]:
class AddNode(object):    
    """
    Adds several inputs nodes.
    """
    def __init__(self, input_nodes):
        self.inputs = input_nodes
        self.output = ValueNode(0.0)

    def forward(self):
        self.output.value = reduce(lambda x1, x2:
                                   x1.value + x2.value if type(x1) == ValueNode else x1 + x2.value,
                                   self.inputs)
    
    def backward(self):
        for n in self.inputs:
            n.gradient = 1 * self.output.gradient

In [5]:
class SigmoidNode(object):
    """
    Adds a sigmoid non-linearity to a single input
    """
    def __init__(self, x):
        self.x = x
        self.output = ValueNode(0.0)
        
    def forward(self):
        self.output.value = 1/(1 + np.exp(-1 * self.x.value))
        
    def backward(self):
        s = 1/(1 + np.exp(-1 * self.x.value))
        self.x.gradient = (s * (1 - s)) * self.output.gradient

In [6]:
class Perceptron(object):
    def __init__(self, input_nodes, alpha=0.001):
        ### Hyper parameters
        self.alpha = alpha
        ### Initializing weights/bias to a random float between -1 and 1.
        self.weights = [ValueNode(np.random.uniform(-1, 1)) for i in range(len(input_nodes))]
        self.b = ValueNode(np.random.uniform(-1, 1))
        ### Input and Output variables
        self.inputs = input_nodes
        
        self.multiply_nodes = [MultiplyNode(*p) for p in zip(self.inputs, self.weights)]
        l = list(map(lambda n: n.output, self.multiply_nodes))
        l.append(self.b)
        self.add_node = AddNode(l)
        self.sigmoid_node = SigmoidNode(self.add_node.output)
        self.output = self.sigmoid_node.output
    
    def forward(self):
        for n in self.multiply_nodes:
            n.forward()
        self.add_node.forward()
        self.sigmoid_node.forward()
        
    def backward(self):
        self.sigmoid_node.backward()
        self.add_node.backward()
        for n in self.multiply_nodes:
            n.backward()
    
    def update(self):
        for n in self.weights:
            n.value -= self.alpha * n.gradient
        self.b.value -= self.alpha * self.b.gradient

### Validation

In [7]:
x1 = ValueNode(0.11)
x2 = ValueNode(-1.0)
p1 = Perceptron([x1, x2], alpha=0.1)
p2 = Perceptron([p1.output], alpha=0.1)

# number of iterations
N = 100000
# expected output 
target = 0.3481972639817

for i in range(N):
    # Step 1. Forward Pass
    p1.forward()
    p2.forward()
    # Step 2. Calculate Loss. -2 * (y - output) is the gradient of output w.r.t 
    # square loss function.
    p2.output.gradient = -2 * (target - p2.output.value)
    # Step 3. Backward Pass
    p2.backward()
    p1.backward()
    # Step 4. Update Weights and Bias
    p2.update()
    p1.update()

p2.output.value

0.3481972639817006